# 第七章 迪克斯特拉算法

# 一、 迪克斯特拉（dijkstras）算法介绍

算法目标：找出一个图中最快（耗时最短）的路径。 

实现步骤：
- 找出最短时间内前往的节点；
- 对于该节点的邻居，检查是否有前往它们的更短路径，如果有，就更新其开销；
- 重复这个过程，直到对图中的每个节点都重复了以上两个步骤；
- 计算最终路径

#### 加权图

迪杰斯特拉算法用于每条边都有关联数字的图，这些数字称为权重（weight）。

带权重的图称为加权图(weighted graph)。权重全为非负数的为正加权图。

**迪克斯特拉算法只适用于权重全为非负数的加权图**。包含负权边的图，需要找最短路径(耗时最短)可以使用 贝尔曼-福德算法。

## 二、算法实现

要解决这类问题，需要三个散列表，graph, costs, parents.
- graph: 同时存储所有节点的邻居和该一节点前往其邻居的开销 （不是从起点到节点A的邻居B的开销，是直接A->B的开销）。(终点的graph设置为空dict)
- costs: 存储从起点到每个节点的开销（起点->A, 起点->B,...）。（从起点出发前往该节点需要的权重，不知道的开销，设置为无穷大float('inf')）
- parents: 存储父节点的散列表。（父节点不是起点的可以直接设置为None, parents['e'] = None）

换钢琴代码：

#### 1. 节点和权重定义 ： 散列表graph
- 所有节点包括起点和终点及其对应的指向邻居的权重

In [1]:
graph = {}                       # graph 需要定义为dict
graph['music'] = {}              # graph['music']需要定义为dict  
graph['music']['album'] = 5
graph['music']['poster'] = 0
graph['album'] = {}              # graph['album']需要定义为dict  
graph['album']['gitar'] = 15
graph['album']['drum'] = 20
graph['poster'] = {}             # graph['poster']需要定义为dict  
graph['poster']['gitar'] = 30
graph['poster']['drum'] = 35
graph['gitar'] = {}              # graph['gitar']需要定义为dict  
graph['gitar']['piano'] = 20
graph['drum'] = {}               # graph['drum']需要定义为dict  
graph['drum']['piano'] = 10
graph['piano'] = {}              # 终节点graph['piano']需要定义为dict  
# 所有节点都一定要用散列表定义

In [2]:
graph

{'album': {'drum': 20, 'gitar': 15},
 'drum': {'piano': 10},
 'gitar': {'piano': 20},
 'music': {'album': 5, 'poster': 0},
 'piano': {},
 'poster': {'drum': 35, 'gitar': 30}}

#### 2. 节点开销定义：散列表costs
- 从**起点**出发前往其他节点的权重（与起点不相邻，则权重设置为无穷大float('inf')）

In [3]:
infinity = float('inf')
costs = {}
costs['album'] = 5
costs['poster'] = 0
costs['gitar'] = infinity
costs['drum'] = infinity
costs['piano'] = infinity

In [4]:
costs

{'album': 5, 'drum': inf, 'gitar': inf, 'piano': inf, 'poster': 0}

#### 3. 父节点定义：散列表parents
- 不是起点邻居的节点的父节点可以直接设置为None.
- 最后确定的最短路径的节点就储存在parents中，但是由于散列表无序，若找最佳节点排序，从终点开始倒推父节点parents[pinao], parents[parents[pinao]],....

In [5]:
parents = {}
parents['album'] = 'music'
parents['poster'] = 'music'
parents['gitar'] = None
parents['drum'] = None
parents['piano'] = None

In [6]:
parents

{'album': 'music',
 'drum': None,
 'gitar': None,
 'piano': None,
 'poster': 'music'}

#### 4. 已处理节点定义：列表or数组
- 记录处理过的节点

In [7]:
processed = []

#### 5. 开销最低节点查找函数
- 遍历未处理的节点，寻找起点出发到节点的开销最低的节点

In [8]:
def find_lowest_cost_node(costs):
    # 初始化最低开销lowest_cost和最低开销节点lowest_cost_node
    lowest_cost = float('inf')
    lowest_cost_node = None
    
    # 遍历所有未处理节点，比较获得最低开销节点
    for node, cost in costs.items():
        if cost < lowest_cost and node not in processed:    # node not in processed： 必须有，未处理节点
            lowest_cost = cost
            lowest_cost_node = node
    return lowest_cost_node

In [9]:
find_lowest_cost_node(costs)

'poster'

#### 6. 算法实现核心：
- 迪克斯特拉算法背后的关键理念：**找出图中最便宜的节点，并确保没有到该节点的更便宜的路径**。

In [10]:
# 第一部：初始化从起点出发到所有节点的开销最低的节点
node = find_lowest_cost_node(costs)
# while 循环，循环结束条件是所有节点都被处理完，即 node = None
while node is not None:
    # 定义开销cost值和对应开销的节点的邻居
    cost = costs[node]
    neighbors = graph[node]
    
    # 遍历当前节点的所有邻居，如果经当前节点前往该邻居更近，则更新开销costs表和父节点parents表
    for n in neighbors.keys():
        new_cost = cost + neighbors[n]
        if costs[n] > new_cost:
            costs[n] = new_cost     # 起点出发经node到n的开销 比 起点出到达n(直达or经停，未知)的开销低，则更新起点到n(直达or经停，未知)的开销值为经停的开销
            parents[n] = node       # 同时，更新n(node的邻居)的父节点(初始化的已知or未知) 为新的父节点node(经停开销最低的节点node)
    
    processed.append(node)          # 当前节点标记为已处理
    
    # 更新 未处理节点中的最低开销节点
    node = find_lowest_cost_node(costs)

In [11]:
parents

{'album': 'music',
 'drum': 'album',
 'gitar': 'album',
 'piano': 'drum',
 'poster': 'music'}

In [12]:
# 最低开销 = costs['piano']
print(costs)
print('The lowest cost from to exchange piano with music:',costs['piano'])

{'album': 5, 'poster': 0, 'gitar': 20, 'drum': 25, 'piano': 35}
The lowest cost from to exchange piano with music: 35


In [13]:
# 开销最低的路径 
print(parents[parents[parents['piano']]],'->',parents[parents['piano']],'->',parents['piano'],'->','piano')

music -> album -> drum -> piano


In [14]:
print(' music->album:',costs['album'], '\n','album->drum:',costs['drum'], '\n','drum->piano:',costs['piano'])

 music->album: 5 
 album->drum: 25 
 drum->piano: 35


## 三、小结

- 广度有限搜索用于在非加权图中查找最短路径。
- 迪杰斯特拉算法用于在加权图中查找最短路径。
- 仅当权重为正时迪杰斯特拉算法才管用。
- 如果图中包含负权边，考虑使用贝尔曼-福德（Bellman-Ford）算法。

## 四、练习

In [15]:
# exercise 7.1
# graph
graph1 = {}
graph1['start'] = {}
graph1['start']['A'] = 5
graph1['start']['B'] = 2
graph1['A'] = {}
graph1['A']['C'] = 4
graph1['A']['D'] = 2
graph1['B'] = {}
graph1['B']['A'] = 8
graph1['B']['D'] = 7
graph1['C'] = {}
graph1['C']['D'] = 6
graph1['C']['final'] = 3
graph1['D'] = {}
graph1['D']['final'] = 1
graph1['final'] = {}              # ‘final’节点不能少
# costs: cost from the start
infinity = float('inf')
costs1 = {}
costs1['A'] = 5
costs1['B'] = 2
costs1['C'] = infinity
costs1['D'] = infinity
costs1['final'] = infinity
# parents
parents1 = {}
parents1['A'] = 'start'
parents1['B'] = 'start'
parents1['C'] = None
parents1['D'] = None
parents1['final'] = None

In [16]:
print(graph1)
print(costs1)
print(parents1)

{'start': {'A': 5, 'B': 2}, 'A': {'C': 4, 'D': 2}, 'B': {'A': 8, 'D': 7}, 'C': {'D': 6, 'final': 3}, 'D': {'final': 1}, 'final': {}}
{'A': 5, 'B': 2, 'C': inf, 'D': inf, 'final': inf}
{'A': 'start', 'B': 'start', 'C': None, 'D': None, 'final': None}


In [17]:
# 已处理过的节点
processed1 = []

In [18]:
# 寻找最小开销cost的节点
def find_lowest(costs):
    lowest_cost = float('inf')
    lowest_cost_node = None
    
    for nodel, cost in costs.items():
        if cost < lowest_cost and nodel not in processed1:
            lowest_cost = cost
            lowest_cost_node = nodel
    return lowest_cost_node

In [19]:
# algorithm
nodel1 = find_lowest(costs1)
while nodel1 is not None:
    # obtain the cost and neighbour info of current node
    cost1 = costs1[nodel1]
    neighbs = graph1[nodel1]
    
    # loop for the neighbs
    for i in neighbs.keys():
        new_cost1 = cost1 + neighbs[i]     # 从起点经nodel1到nodel1的邻居i的开销
        if new_cost1 < costs1[i]:          # 从起点经nodel1到nodel1的邻居i的开销  与 初始定义的 从起点到nodel1的开销(位置是否经停) 比较
            costs1[i] = new_cost1           # 数值小的重新赋值给costs1 即更新costs1表
            parents1[i] = nodel1             # 更新parents表，值小的nodel1作为i的父节点
    
    processed1.append(nodel1)
    nodel1 = find_lowest(costs1)

In [20]:
parents1

{'A': 'start', 'B': 'start', 'C': 'A', 'D': 'A', 'final': 'D'}

In [21]:
# 从起点到终点的最短路径的总权重：
costs1['final']

8

In [22]:
# 路径
print(parents1[parents1[parents1['final']]],'->',parents1[parents1['final']],'->',parents1['final'],'->','final')

start -> A -> D -> final


In [23]:
costs1

{'A': 5, 'B': 2, 'C': 9, 'D': 7, 'final': 8}